In [1]:
# %%time
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.metrics import mean_squared_error,mean_absolute_error,r2_score
import warnings
warnings.filterwarnings(action = 'ignore')
from sklearn.metrics import r2_score, accuracy_score
from xgboost import XGBRegressor

In [2]:
data=pd.read_csv('Cars24.csv')
data.head()

,Unnamed: 0,Car Brand,Model,Price,Model Year,Location,Fuel,Driven (Kms),Gear,Ownership,EMI (monthly)
0,0,Hyundai,EonERA PLUS,330399,2016,Hyderabad,Petrol,10674,Manual,2,7350
1,1,Maruti,Wagon R 1.0LXI,350199,2011,Hyderabad,Petrol,20979,Manual,1,7790
2,2,Maruti,Alto K10LXI,229199,2011,Hyderabad,Petrol,47330,Manual,2,5098
3,3,Maruti,RitzVXI BS IV,306399,2011,Hyderabad,Petrol,19662,Manual,1,6816
4,4,Tata,NanoTWIST XTA,208699,2015,Hyderabad,Petrol,11256,Automatic,1,4642


In [3]:
a=1

input = pd.DataFrame({
    "Car Brand"     : ["Maruti"],  # String
    "Model"         : ["Wagon R 1.0LXI"],  # String
    "Price"         : [0],  # int
    "Model Year"    : [2011],  # int
    "Location"      : ["Hyderabad"],  # String
    "Fuel"          : ["Petrol"],  # String
    "Driven (Kms)"  : [20979],  # int
    "Gear"          : ["Manual"],  # String
    "Ownership"     : [1],  # int
})

In [4]:
# output = data.tail(a)
data.drop(data[data.Price>1500000].index, axis=0, inplace=True)
data = data.drop(['Unnamed: 0','EMI (monthly)'],axis=1)

data = data.dropna(axis=0)
data = data.append(input, ignore_index=True)
output = data.tail(a)

data=pd.get_dummies(data, columns=["Location",'Fuel','Car Brand'],drop_first=True)
# data['Ownership']=max(data['Ownership'])-data['Ownership']+1

encoder = LabelEncoder()
objects = []
for i in data.columns:
    if data[i].dtype == 'object':
        objects.append(i)
encoder = LabelEncoder()
encoder_list = []
for i in objects:
    encoder.fit(data[i])
    encoder_list.append(encoder.classes_)
    data[i] = encoder.transform(data[i])
data['Model Year'] = 2021 - data['Model Year']

x=data.drop(['Price'],axis=1)
y=data['Price']
x_testing = x.iloc[-a:]

x.drop(x.tail(a).index, inplace=True)
y.drop(x.tail(a).index,inplace=True)

x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2,random_state=0)

model = XGBRegressor(max_depth=5,max_leaves=2,n_estimators=800, learning_rate=0.2)
model.fit(x_train,y_train)
cvs = cross_val_score(model,x_train, y_train, cv=10)
#     print("Score: ",cvs)
#     print('-'*30)
print("Validation Score Mean", cvs.mean())
predict = model.predict(x_test)
result = {'Actual':y_test,'Predict':predict, 'Residual':y_test-predict}
result = pd.DataFrame(result)
result['Predict'] = result['Predict'].astype('int64')
result['Res%'] = (result['Residual']/result['Predict']).round(2)
output['Price'] = model.predict(x_testing).astype('float')
# print(output,result)
# print(result.head(10))
print('r2_score:',r2_score(predict, y_test))

Validation Score Mean 0.8914710931963207
r2_score: 0.8912620600093495


In [5]:
result

,Actual,Predict,Residual,Res%
15,234999,267277,-32278.84375,-0.12
348,273799,271772,2026.09375,0.01
3449,635000,636874,-1874.43750,-0.00
5540,1234899,1135458,99440.62500,0.09
4150,929899,900434,29465.00000,0.03
...,...,...,...,...
4114,854499,956000,-101501.06250,-0.11
3972,312199,274221,37977.46875,0.14
1529,334899,363083,-28184.15625,-0.08
624,289799,223382,66416.65625,0.30


In [6]:
output

,Car Brand,Model,Price,Model Year,Location,Fuel,Driven (Kms),Gear,Ownership
5545,Maruti,Wagon R 1.0LXI,344257.90625,2011,Hyderabad,Petrol,20979,Manual,1
